In [1]:
def nextBaseStep(fromIdx, toIdx, baseAggr=100):
    # get next loan idx that is closest in terms of mod baseAggr
    nextIdx = (fromIdx // baseAggr + 1) * baseAggr
    # make sure next doesn't "overstep" toIdx
    return min([nextIdx, toIdx])

def buildBaseAggrRanges(loanIdxsWhereSharesChanged, baseAggr=100):
    baseAggrRanges = loanIdxsWhereSharesChanged.copy()
    i = 0
    n = len(baseAggrRanges) - 1
    while (i < n):
        # get next loan idx in terms of mod baseAggr
        nextStep = nextBaseStep(baseAggrRanges[i], baseAggrRanges[i+1], baseAggr)
        # make sure we don't insert duplicates
        if (nextStep != baseAggrRanges[i+1]):
            baseAggrRanges.insert(i+1, nextStep)
            n = len(baseAggrRanges) - 1
        i += 1
    return baseAggrRanges

def prune(baseRanges, pruneBy=1000):
    pruned = [baseRanges[0]]
    i = 0
    n = len(baseRanges)
    while (i < n):
        # make sure we don't insert duplicates
        if pruned[-1] != baseRanges[i]:
            pruned.append(baseRanges[i])
        # check if loan idx is potentially starting point for range that can be pruned
        if baseRanges[i] % pruneBy == 0 and i + 10 < n:
            canBePruned = True
            for j in range(10):
                # check that next next 10 loan idxs can all be pruned
                if baseRanges[i+j+1] % (pruneBy/10) != 0:
                    canBePruned = False
                    break
            # if range can be pruned, add the 10th item and move forward by 10
            if canBePruned:
                pruned.append(baseRanges[i+j+1])
                i += 10
            else:
                i += 1
        else:
            i += 1
    return pruned

def findBreakPoints(loanIdxsWhereSharesChanged, baseRanges):
    breakPoints = []
    i = 0
    j = 0
    n = len(baseRanges)
    while (i < n):
        if baseRanges[i] == loanIdxsWhereSharesChanged[j]:
            breakPoints.append(i)
            j += 1
        i += 1
    return breakPoints
        
def pruneBreakPointAware(baseRanges, pruningBreakPoints):
    # if no break points, prune entire range
    if len(pruningBreakPoints) == 1:
        # try pruning by 1'000 intervals
        prunedBy1000 = prune(baseAggrRanges, 1000)
        
        # try pruning by 10'000 intervals
        pruned = prune(prunedBy1000, 10000)

    # if break points, prune subintervals
    else:
        pruned = []
        for i in range(len(pruningBreakPoints)-1):
            # get range to be pruned
            rangeToBePruned = baseAggrRanges[pruningBreakPoints[i]:(pruningBreakPoints[i+1]+1)]
            
            # try pruning by 1'000 intervals
            prunedBy1000 = prune(rangeToBePruned, 1000)
            
            # try pruning by 10'000 intervals
            prunedBy10000 = prune(prunedBy1000, 10000)
            
            if i == 0:
                pruned = prunedBy10000
            else:
                pruned += prunedBy10000[1:]
    return pruned
    
def packIntoClaimingTxs(pruned):
    packed = []
    i = 0
    n = len(pruned)-1
    while (i < n):
        consecutive100 = pruned[i] % 100 == 0 and pruned[i+1] % 100 == 0
        consecutive1000 = pruned[i] % 1000 == 0 and pruned[i+1] % 1000 == 0
        consecutive10000 = pruned[i] % 10000 == 0 and pruned[i+1] % 10000 == 0
        if (consecutive100 or consecutive1000 or consecutive10000):
            j = 1
            tmp1 = [pruned[i]]
            tmp2 = [] # tmp variable to keep track of "closing" uneven upper claiming bound, e.g., 100,200,255 -> 100,200 + 200,255
            while (i+j < n+1):
                diff = 10000 if consecutive10000 else (1000 if consecutive1000 else 100)
                if (pruned[i+j] - pruned[i]) % diff == 0:
                    tmp1.append(pruned[i+j])
                    j += 1
                else:
                    tmp2 = [pruned[i+j-1], pruned[i+j]]
                    break
            i += j
            packed.append(tmp1)
            if len(tmp2) > 0:
                packed.append(tmp2)
        else:
            packed.append([pruned[i], pruned[i+1]])
            i += 1
    return packed
           
def packBreakPointAware(pruned, packingBreakPoints):
    # if no break points, pack entire range
    if len(packingBreakPoints) == 1:
        packed = packIntoClaimingTxs(pruned)
    else:
        packed = []
        for i in range(len(packingBreakPoints)-1):
            rangeToBePacked = pruned[packingBreakPoints[i]:(packingBreakPoints[i+1]+1)]
            packed.append(packIntoClaimingTxs(rangeToBePacked))
    return packed
            
# example array of where LP changed position
loanIdxsWhereSharesChanged = [4,1300,1555,2600, 10000]
print("loanIdxsWhereSharesChanged:\n", loanIdxsWhereSharesChanged, "\n")

# construct array with "intermediate" steps according to baseAggr
baseAggrRanges = buildBaseAggrRanges(loanIdxsWhereSharesChanged, 100)
print("baseAggrRanges:\n", baseAggrRanges, "\n")

# find break points across which pruning shall not happen
pruningBreakPoints = findBreakPoints(loanIdxsWhereSharesChanged, baseAggrRanges)
print("pruningBreakPoints:\n", pruningBreakPoints, "\n")
     
# prune taking into account break points
pruned = pruneBreakPointAware(baseAggrRanges, pruningBreakPoints)
print("pruned:\n", pruned, "\n")

# find break points across which tx packing shall not happen
packingBreakPoints = findBreakPoints(loanIdxsWhereSharesChanged, pruned)
print("packingBreakPoints:\n", packingBreakPoints, "\n")

# get packed claimings taking into account break points
packed = packBreakPointAware(pruned, packingBreakPoints)
print("packed:\n", packed, "\n")

loanIdxsWhereSharesChanged:
 [4, 1300, 1555, 2600, 10000] 

baseAggrRanges:
 [4, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1555, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800, 9900, 10000] 

pruningBreakPoints:
 [0, 13, 16, 27, 101] 

pruned:
 [4, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1555, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000] 

packingBreakPoints:
 [0, 13, 16, 27, 38]